In [85]:
!pip install -U transformers

In [86]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [87]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [88]:
examples = [
    ("The movie was fantastic and full of thrilling moments!", "Positive"),
    ("The plot was boring and the acting was terrible.", "Negative"),
    ("Absolutely loved the soundtrack and the performances!", "Positive"),
]

In [89]:
def build_prompt(review, shots=1):
    prompt = "Classify the sentiment of the following reviews as Positive or Negative.\n\n"
    for i in range(min(shots, len(examples))):
        ex_text, ex_label = examples[i]
        prompt += f"Review: {ex_text}\nSentiment: {ex_label}\n\n"

    prompt += f"Review: {review}\nSentiment:"
    return prompt

In [90]:
def classify_review(review, shots=1, max_new_tokens=10):
    prompt = build_prompt(review, shots)
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_text = generated_text[len(prompt):].strip().lower()

    if "positive" in pred_text:
        return "Positive"
    elif "negative" in pred_text:
        return "Negative"
    else:
        if pred_text.startswith("p"):
            return "Positive"
        elif pred_text.startswith("n"):
            return "Negative"
        else:
            return "Negative"

In [91]:
reviews = [
    "I loved the movie, it was fantastic!",
    "The movie was amazing and full of surprises!",
    "The soundtrack was beautiful and inspiring.",
    "The plot was boring and uninteresting."
]

In [92]:
for r in reviews:
    print("------")
    for s in [1, 2, 3]:
        print(f"{s}-shot:", classify_review(r, shots=s))

------
1-shot: Negative
2-shot: Negative
3-shot: Negative
------
1-shot: Negative
2-shot: Negative
3-shot: Negative
------
1-shot: Negative
2-shot: Negative
3-shot: Negative
------
1-shot: Negative
2-shot: Negative
3-shot: Negative
